# Load dataset Daily CHIRPS

## Load files

In [49]:
import os
path = 'C:/Users/manue/OneDrive/Documentos/phd/data/chirps/'
files = os.listdir(path)

## Establish the geographic points of Boyacá

In [88]:
import numpy as np
x = np.arange(-74.8, -71.9, 0.05)
y = np.arange(4.5, 7.3, 0.05)
xx, yy = np.meshgrid(x, y)
xnorm = (xx - xx.min()) / (xx.max() - xx.min())
ynorm = (yy - yy.min()) / (yy.max() - yy.min())

## Create dataset CHIRPS precipitation data

In [130]:
import netCDF4 as nc
import xarray as xr
import pandas as pd
for file in files:
        ds_chirps_nc = xr.open_dataset(xr.backends.NetCDF4DataStore(nc.Dataset(path+file)))
        # delimit the boundaries of Boyacá
        ds_boyaca = ds_chirps_nc.sel(longitude=slice(-74.8,-71.9), latitude=slice(4.5,7.3))
        df_all = pd.concat([df_all,ds_boyaca.to_dataframe().reset_index()])

In [131]:
df_all

,latitude,longitude,time,precip
0,4.524998,-74.775002,2022-01-01,15.852234
1,4.524998,-74.775002,2022-01-02,0.000000
2,4.524998,-74.775002,2022-01-03,0.000000
3,4.524998,-74.775002,2022-01-04,0.000000
4,4.524998,-74.775002,2022-01-05,0.000000
...,...,...,...,...
1185515,7.274998,-71.925003,2022-12-27,0.000000
1185516,7.274998,-71.925003,2022-12-28,0.000000
1185517,7.274998,-71.925003,2022-12-29,0.000000
1185518,7.274998,-71.925003,2022-12-30,5.989796
